In [1]:
from yahoo_fin import stock_info as si
import yfinance as yf
import pandas as pd
import os.path

class price:
    def __init__(self, ticker, time='max'):
        self.stock = yf.Ticker(ticker)
        self.info = self.stock.info
        #period : 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        self.price = self.stock.history(period=time)

class fundamental(price):
    def __init__(self, ticker, time='max', interval = 'year', level_of_rounding = 'million'):
        try:
            super().__init__(ticker, time)
            print("Please Wait...")
            
            if level_of_rounding == 'million':
                rounding = 1000000
            elif level_of_rounding == 'billion':
                rounding = 1000000000
            elif level_of_rounding == 'one':
                rounding = 1
            
            if interval == 'year':
                year_statement = True
            elif interval == 'quarter':
                year_statement = False
                
            #get data from Yahoo Finance
            self.balance_sheet_raw = si.get_balance_sheet(ticker,year_statement)
            self.income_statement_raw = si.get_income_statement(ticker, year_statement)
            self.cash_flow_raw = si.get_cash_flow(ticker, year_statement)
            self.valuation = si.get_stats_valuation(ticker)
            #reshaping and pivoting
            self.total_share = int(self.info['marketCap']/self.price.iloc[-1,3])/rounding
            balance_sheet = self.balance_sheet_raw.loc[['totalAssets', 'totalStockholderEquity', 'retainedEarnings', 'totalLiab']]/rounding
            balance_sheet = balance_sheet.reset_index()
            self.balance_sheet = balance_sheet.rename(columns={'Breakdown': 'Keys'})
            income_statement = self.income_statement_raw.loc[['totalRevenue', 'operatingIncome', 'netIncome']]/rounding
            income_statement = income_statement.reset_index()
            self.income_statement = income_statement.rename(columns={'Breakdown': 'Keys'})
            cash_flow = self.cash_flow_raw.loc[['totalCashflowsFromInvestingActivities', 'totalCashFromFinancingActivities', 'totalCashFromOperatingActivities']]/rounding
            cash_flow = cash_flow.reset_index()
            self.cash_flow = cash_flow.rename(columns={'Breakdown': 'Keys'})
            self.financial = self.balance_sheet.append([self.income_statement, self.cash_flow]).set_index('Keys').T
            #add ratio
            self.financial['debtEquityRatio'] = self.financial['totalLiab']/self.financial['totalStockholderEquity']
            self.financial['retainedEarningsEquityRatio'] = self.financial['retainedEarnings']/self.financial['totalStockholderEquity']
            self.financial['returnOnEquity'] = self.financial['netIncome']/self.financial['totalStockholderEquity']
            self.financial['operationalProfitMargin'] = self.financial['operatingIncome']/self.financial['totalRevenue']
            self.financial['netProfitMargin'] = self.financial['netIncome']/self.financial['totalRevenue']
            self.financial['earningPerShare'] = self.financial['netIncome']/self.total_share
            self.financial['bookValuePerShare'] = self.financial['totalStockholderEquity']/self.total_share
            print("Done")
        except:
            print('Error')
            pass

def write_csv(filename, df):
    if os.path.isfile(filename):
        df.to_csv(filename, header=False, index=False, mode='a')
    else:    
        df.to_csv(filename, header=True, index=False, mode='w')

def add_data(file, dataframe, ticker):
    dataframe['stockCode'] = ticker
    stock_columns = dataframe.columns.values.tolist()
    list_columns = [stock_columns[-1]] + stock_columns[0:-1]
    dataframe = dataframe.reindex(columns= list_columns)
    write_csv(file, dataframe)
                     
def add_stock_data(dataframe, fundamental_file, price_file, time = '5y', interval = 'year', no = 1):
    number = no - 1           
    for stock_ticker in dataframe['Kode'].iloc[number:]:
        print(number+1, 'Downloading Data :', stock_ticker)
        number += 1
        stock_data = fundamental(stock_ticker + '.JK', time, interval)
        #fundamental and price
        stock_value = stock_data.financial.T.reset_index()
        stock_price = stock_data.price.reset_index()[['Date', 'Close']]
        add_data(fundamental_file, stock_value, stock_ticker)
        add_data(price_file, stock_price, stock_ticker)

In [2]:
#read excel
# jii70 = pd.read_excel('JII70.xlsx', engine = 'openpyxl', header=0)

In [3]:
# add_stock_data(jii70, 'JII70_fundamental_quarter.csv', 'JII70_price_quarter.csv', time='1y', interval='quarter')

In [40]:
data_stock = pd.read_csv('JII70_fundamental_quarter.csv')
data_price = pd.read_csv('JII70_price_quarter.csv')

In [43]:
ticker = 'AALI'
stock = data_stock.loc[data_stock['stockCode'] == ticker]
stock = stock.drop(columns = 'stockCode').T
stock.columns = stock.iloc[0]
stock = stock.drop('Keys')
stock = stock.set_index([pd.Index([3, 2, 1, 4])])
stock = stock.iloc[::-1]
stock = stock.reset_index()
stock = stock.rename(columns = {'index': 'Quarter'})
stock

Keys,Quarter,totalAssets,totalStockholderEquity,retainedEarnings,totalLiab,totalRevenue,operatingIncome,netIncome,totalCashflowsFromInvestingActivities,totalCashFromFinancingActivities,totalCashFromOperatingActivities,debtEquityRatio,retainedEarningsEquityRatio,returnOnEquity,operationalProfitMargin,netProfitMargin,earningPerShare,bookValuePerShare
0,4,2.69741e+07,1.85212e+07,1.38439e+07,7.9956e+06,5.06626e+06,609658,99934,-383236,-192458,487333,0.431701,0.747463,0.00539567,0.120337,0.0197254,51.9221,9622.93
1,1,2.92186e+07,1.88963e+07,1.42149e+07,9.85495e+06,4.79608e+06,639614,371063,-188439,440855,660610,0.521528,0.752261,0.0196368,0.133362,0.0773679,192.791,9817.83
2,2,2.73813e+07,1.88683e+07,1.41415e+07,8.06191e+06,4.28493e+06,116587,20838,-192882,-828150,913217,0.427274,0.749485,0.00110439,0.0272086,0.00486309,10.8267,9803.27
3,3,2.81138e+07,1.89247e+07,1.42513e+07,8.71851e+06,4.24273e+06,341442,190648,-235943,-197443,1.00365e+06,0.460695,0.753052,0.010074,0.080477,0.0449353,99.0539,9832.6


In [ ]:
import datetime
price.loc[:,'Date'] = price['Date'].astype('datetime64[ns]')
price1 = price[price['Date'].dt.year == '2016']
price = data_price.loc[data_price['stockCode'] == 'ULTJ']
# price1['per'] = price['Close']
# price1['pbv'] = price['Close']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ticker = 'AALI'
time_interval = 'Quarter'
index_list = [3, 2, 1, 4]
stock = data_stock.loc[data_stock['stockCode'] == ticker]
stock_ticker = yf.Ticker(ticker+'.JK')
stock_info = stock_ticker.info
rounding = 1000000

stock = stock.drop(columns = 'stockCode').T
stock.columns = stock.iloc[0]
stock = stock.drop('Keys')
stock = stock.set_index([pd.Index(index_list)])
stock = stock.iloc[::-1]
stock = stock.reset_index()
stock = stock.rename(columns = {'index': time_interval})

sns.set(style="darkgrid")
sns.set_color_codes("pastel")

fig = plt.figure(figsize=(10,14))
fig.suptitle(stock_info['longName']+' ('+ticker+')', fontsize=16)

plt.subplot2grid((5, 2), (0, 0), colspan=2)
plt.bar(stock[time_interval]+0.3, stock['totalAssets']/rounding, width=0.2, color='b', label='Asset')
plt.bar(stock[time_interval]+0.1, stock['totalStockholderEquity']/rounding, width=0.2, color='g', label='Equity')
plt.bar(stock[time_interval]-0.1, stock['retainedEarnings']/rounding, width=0.2, color='r', label='Retained Earning')
plt.bar(stock[time_interval]-0.3, stock['totalLiab']/rounding, width=0.2, color='y', label='Liabilities')
plt.xticks(index_list)
plt.ylabel('Value (Trillion)')
plt.title('Balance Sheet')
plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

plt.subplot2grid((5, 2), (1, 0), colspan=2)
plt.bar(stock[time_interval]-0.2, stock['totalRevenue']/rounding, width=0.2, color='b', label='Revenue')
plt.bar(stock[time_interval], stock['operatingIncome']/rounding, width=0.2, color='g', label='Operating Income')
plt.bar(stock[time_interval]+0.2, stock['netIncome']/rounding, width=0.2, color='r', label='Net Income')
plt.xticks(index_list)
plt.ylabel('Value (Trillion)')
plt.title('Income Statement')
plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

plt.subplot2grid((5, 2), (2, 0), colspan=2)
plt.bar(stock[time_interval]-0.2, stock['totalCashflowsFromInvestingActivities']/rounding, width=0.2, color='b', label='Investing')
plt.bar(stock[time_interval], stock['totalCashFromFinancingActivities']/rounding, width=0.2, color='g', label='Financing')
plt.bar(stock[time_interval]+0.2, stock['totalCashFromOperatingActivities']/rounding, width=0.2, color='r', label='Operating')
plt.xticks(index_list)
plt.ylabel('Value (Trillion)')
plt.title('Cash Flow')
plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

plt.subplot2grid((5, 2), (3, 0))
plt.bar(stock[time_interval], stock['earningPerShare'], width=0.2, color='b', label='EPS')
plt.xticks(index_list)
plt.legend(bbox_to_anchor=(-0.12, 1), loc='upper right')

plt.subplot2grid((5, 2), (3, 1))
plt.bar(stock[time_interval], stock['bookValuePerShare'], width=0.2, color='g', label='BVPS')
plt.xticks(index_list)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.subplot2grid((5, 2), (4, 0))
plt.bar(stock[time_interval]-0.2, stock['debtEquityRatio'], width=0.2, color='b', label='DER')
plt.bar(stock[time_interval], stock['retainedEarningsEquityRatio'], width=0.2, color='g', label='RER')
plt.xticks(index_list)
plt.legend(bbox_to_anchor=(-0.12, 1), loc='upper right')

plt.subplot2grid((5, 2), (4, 1))
plt.bar(stock[time_interval]+0.2, stock['operationalProfitMargin'], width=0.2, color='b', label='OPM')
plt.bar(stock[time_interval], stock['netProfitMargin'], width=0.2, color='g', label='NPM')
plt.bar(stock[time_interval]-0.2, stock['returnOnEquity'], width=0.2, color='r', label='ROE')
plt.xticks(index_list)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig(ticker+'_quarter.png')

In [48]:
stock

Keys,Year,totalAssets,totalStockholderEquity,retainedEarnings,totalLiab,totalRevenue,operatingIncome,netIncome,totalCashflowsFromInvestingActivities,totalCashFromFinancingActivities,totalCashFromOperatingActivities,debtEquityRatio,retainedEarningsEquityRatio,returnOnEquity,operationalProfitMargin,netProfitMargin,earningPerShare,bookValuePerShare
0,2016,2.99815e+07,1.84088e+07,9.66117e+06,1.15727e+07,9.10626e+06,80015.9,64810.3,-1.38205e+06,-26840.1,1.01539e+06,0.628653,0.524814,0.00352062,0.00878691,0.00711712,13.4812,3829.21
1,2017,3.00143e+07,1.84904e+07,9.76572e+06,1.15239e+07,1.26536e+07,600606,136507,-2.89056e+06,-568358,1.37918e+06,0.623236,0.528151,0.00738258,0.0474652,0.010788,28.3948,3846.19
2,2018,3.21954e+07,1.84483e+07,8.14838e+06,1.3747e+07,2.52752e+07,1.84613e+06,1.636e+06,-2.59381e+06,-619760,1.87458e+06,0.745161,0.441686,0.0886801,0.0730411,0.0647274,340.305,3837.44
3,2019,3.01949e+07,1.81334e+07,7.91278e+06,1.20615e+07,3.27185e+07,1.00208e+06,193851,-885313,-1.36316e+06,1.63384e+06,0.665153,0.436365,0.0106903,0.0306273,0.00592481,40.323,3771.93
